In [ ]:
import os
import json
import csv
import time
import requests
from datetime import datetime, timedelta

# --- Configuration ---
API_KEY = "RGAPI-dad34b87-7c63-4068-becf-6ff3d8c6605a"
REGION = "na1"
REGIONAL_ENDPOINT = "americas"
TIERS = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND"]
DIVISIONS = ["IV", "III", "II", "I"]

# --- Rate limiting setup ---
REQUEST_LIMIT = 100
WINDOW_SECONDS = 120
request_timestamps = []

# --- State and CSV files ---
STATE_FILE = "state.json"
CSV_FILE = "match_ids.csv"

# --- In-memory cache for summoner PUUIDs ---
puuid_cache = {}

# --- Utility functions for state management ---
def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r") as f:
            return json.load(f)
    return {}

def save_state(state):
    with open(STATE_FILE, "w") as f:
        json.dump(state, f, indent=4)

# --- Rate limiting ---
def rate_limit_sleep():
    global request_timestamps
    now = datetime.now()
    # Remove timestamps older than the time window
    request_timestamps = [t for t in request_timestamps if t > now - timedelta(seconds=WINDOW_SECONDS)]
    if len(request_timestamps) >= REQUEST_LIMIT:
        oldest = request_timestamps[0]
        sleep_time = (oldest + timedelta(seconds=WINDOW_SECONDS)) - now
        print(f"Rate limit reached. Sleeping {sleep_time.total_seconds():.1f}s")
        time.sleep(max(sleep_time.total_seconds(), 0))
    request_timestamps.append(now)

# --- API call with iterative retry ---
def make_api_call(url):
    while True:
        rate_limit_sleep()
        try:
            response = requests.get(url, headers={"X-Riot-Token": API_KEY})
            if response.status_code == 429:
                retry_after = int(response.headers.get("Retry-After", 10))
                print(f"429 Received. Sleeping {retry_after}s")
                time.sleep(retry_after)
                continue  # Retry the request
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"API Error: {str(e)}")
            return None

# --- API helper functions ---
def get_players(tier, division, page=1):
    url = f"https://{REGION}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    return make_api_call(url)

def get_puuid(summoner_id):
    if summoner_id in puuid_cache:
        return puuid_cache[summoner_id]
    url = f"https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
    data = make_api_call(url)
    puuid = data.get("puuid") if data else None
    puuid_cache[summoner_id] = puuid
    return puuid

def get_matches(puuid, count=10):
    url = f"https://{REGIONAL_ENDPOINT}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&count={count}"
    return make_api_call(url) or []

def save_match(match_id, tier, division):
    path = f"matches/{match_id}.json"
    if os.path.exists(path):
        return False  # Match already saved
    url = f"https://{REGIONAL_ENDPOINT}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    data = make_api_call(url)
    if data:
        with open(path, "w") as f:
            json.dump(data, f)
        save_match_id_csv(match_id, tier, division)
        return True
    return False

def save_match_id_csv(match_id, tier, division):
    file_exists = os.path.exists(CSV_FILE)
    with open(CSV_FILE, "a", newline="") as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(["match_id", "tier", "division"])  # Header row
        writer.writerow([match_id, tier, division])

# --- Main data collection function ---
def collect_data(matches_per_tier=2):
    os.makedirs("matches", exist_ok=True)
    state = load_state()  # Load progress state

    for tier in TIERS:
        for division in DIVISIONS:
            print(f"\nProcessing {tier} {division}")
            collected = 0
            page = state.get(tier, {}).get(division, 1)
            done = False

            while collected < matches_per_tier and not done:
                players = get_players(tier, division, page)
                if not players:
                    break

                for player in players:
                    if collected >= matches_per_tier:
                        done = True
                        break
                    puuid = get_puuid(player["summonerId"])
                    if not puuid:
                        continue
                    matches = get_matches(puuid, count=10)
                    for match_id in matches[:5]:
                        if collected >= matches_per_tier:
                            done = True
                            break
                        if save_match(match_id, tier, division):
                            collected += 1
                            print(f"Collected {collected}/{matches_per_tier} for {tier} {division}")
                # Save progress for this tier/division after each page
                state.setdefault(tier, {})[division] = page
                save_state(state)
                page += 1
                time.sleep(0.5)  # Buffer between page requests

            # If we've just finished processing Diamond I, stop the entire collection.
            if tier == "DIAMOND" and division == "I":
                print("Finished Diamond I. Stopping further processing.")
                return

if __name__ == "__main__":
    collect_data()
